In [1]:
import torch
import torchtext
import torch.functional as F
import torch.nn as nn
import spacy
import pandas as pd
import numpy
import random
import time
from torchtext.legacy.data import LabelField, Field, TabularDataset, BucketIterator
SEED=42
torch.manual_seed(SEED)

In [2]:
# path = r"C:\Users\win10\Downloads\Datasets\HS Dataset\labeled_data.csv"
# df = pd.read_csv(path)

# subset = {'tweet':[],'label':[]}
# hcount,ocount, ncount = 0,0,0
# for tweet, label in zip(df['tweet'],df['class']):
#     if(label == 0 and hcount<=1400):
#         hcount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 1 and ocount<=1400):
#         ocount += 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)
#     elif(label == 2 and ncount<=1400):
#         ncount+= 1
#         subset['tweet'].append(tweet)
#         subset['label'].append(label)

# dataset = pd.DataFrame(data = subset, columns = ['tweet','label'])
# dataset.head(10)

# dataset = dataset.sample(frac=1).reset_index(drop=True)

# dataset.head(10)

# dataset.to_csv('dataset.csv')

In [3]:
tokenize = lambda x: x.split()
tweet = Field(sequential=True, use_vocab=True, tokenize=tokenize, include_lengths=True, lower=True)
class_label = LabelField()

In [4]:
fields = {'tweet': ('text',tweet), 'label':('c', class_label)}

In [5]:
path = r"dataset.csv"
data= TabularDataset(path=path, format = 'CSV', fields = fields)

In [6]:
i = random.randint(0,4200)
data.examples[i].text, data.examples[i].c

(['@intheovenjews',
  '@slavecatcher88',
  '@marylene58',
  'we',
  'should',
  'hang',
  'out',
  'ovenjew.',
  'have',
  'a',
  'lot',
  'in',
  'common.',
  'people',
  'waste',
  'time',
  'on',
  'nigs.',
  'jews',
  'r',
  'r',
  'problem.'],
 '0')

In [7]:
state = random.getstate()
train_data, test_data = data.split(split_ratio=0.85, random_state=state)

In [8]:
train_data[654].text

['@danram70',
 'pussy.',
 "it's",
 'science.',
 'donaire',
 'via',
 'rachel',
 'spelling',
 'error']

In [9]:
tweet.build_vocab(train_data, max_size = 10000, min_freq = 2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
class_label.build_vocab(train_data)
print(class_label.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1, '2': 2})


In [11]:
train_iterator, test_iterator = BucketIterator.splits(
    (train_data,test_data),
    batch_size = 8,
    sort = False,
    device = 'cuda')

In [12]:
# for batch in train_iterator:
#     print(batch.c)

In [13]:
tweet.build_vocab(train_data)
tweet.vocab.load_vectors('glove.6B.300d')
embedding = tweet.vocab.vectors.to(device)

In [14]:
batch = next(iter(train_iterator))

In [15]:
for i,batch in enumerate(train_iterator):
    c = batch.c
    a,b = batch.text
    print(i,a,b,c)
    break
#     if(i==2):
#         break

0 tensor([[ 3597,  6896,  6695,  4572,    13,  1468,   886,  2366],
        [   33,     4,  7590,    35,  7659,  1480,    92,  7732],
        [  179,   282,  7524,    33,  1714,  6034,     5,  7746],
        [  188,   411,    27,     2,  7931,  1135,   164,   476],
        [    2,  8043, 12500,   152,     9,   241,    17,    45],
        [ 2418,    43,  9745,   462,    40,   357,     3,    63],
        [   16,   561,  9760,     1,     2,    62,   809,   141],
        [    3,     4,  9341,     1,  3153,    52, 12929, 10528],
        [13428,  8354,     3,     1,    77,     1,   181,   103],
        [    7,    10,   232,     1,     2,     1,  9677,  1076],
        [   95, 12258,   484,     1,  9154,     1,  2679,    45],
        [  162,  8065, 14288,     1,  7032,     1,     8,    41],
        [   55,     7,  8120,     1,  6941,     1,     1,  1507],
        [   35,   142,    60,     1,  6636,     1,     1,    19],
        [   11,  1283,   739,     1,  6252,     1,     1,  1143],
        

In [16]:
embedding.shape

torch.Size([14876, 300])

In [17]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.word_embeddings = nn.Embedding(num_embeddings=47617, embedding_dim=300).from_pretrained(embedding_weights)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
#         self.act = nn.Sigmoid()
 
    def forward(self, sentence, src_len, train = True):
        embeds = self.word_embeddings(sentence)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embeds, src_len.cpu(), enforce_sorted=False)
        packed_outputs, (hidden,cell) = self.lstm(packed_embedded)
#         hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
#         print(hidden.shape)
        dense_outputs = self.fc(hidden)
        outputs=dense_outputs
        return outputs

In [18]:
BATCH_SIZE = 8
nlabel = 3
hidden_dim = 30

model = LSTMClassifier(embedding_dim=embedding.shape[1],hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=embedding)
model = model.to(device)
 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_function = nn.CrossEntropyLoss()
 
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [19]:
from tqdm import tqdm
epochs=10
for epoch in range(epochs):
    time.sleep(1)
    total_loss = 0.0
    total_acc=0.0
    for i, batch in enumerate(tqdm(train_iterator)):
        (feature, batch_length), label = batch.text, batch.c

        optimizer.zero_grad()
        output = model(feature, batch_length).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc+=acc.item() 

    print(f"loss on epoch {epoch+1} = {total_loss/len(train_iterator)}")
    print(f"accuracy on epoch {epoch+1} = {total_acc/len(train_iterator)}")

100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:03<00:00, 133.38it/s]


loss on epoch 1 = 0.8747795806901834
accuracy on epoch 1 = 0.5854026845904271


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 154.91it/s]


loss on epoch 2 = 0.6219372792665324
accuracy on epoch 2 = 0.7505033557580355


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:03<00:00, 143.71it/s]


...

loss on epoch 13 = 0.10545743963268629
accuracy on epoch 13 = 0.9661633109619687


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 156.09it/s]


loss on epoch 14 = 0.07355293066744012
accuracy on epoch 14 = 0.9798657718120806


100%|███████████████████████████████████████████████████████████████████████████████| 447/447 [00:02<00:00, 161.43it/s]

loss on epoch 15 = 0.061286374211797834
accuracy on epoch 15 = 0.9812639821029083


In [20]:
m = nn.LogSoftmax(dim = 1)
_, idx = torch.max(m(output[0].reshape(1,3)), dim=1)
idx

tensor([1], device='cuda:0')

In [21]:
output

tensor([[-2.2706,  1.3862,  0.3607],
        [ 5.3656, -1.9460, -5.0985],
        [-3.3283,  6.7847, -5.1253],
        [ 1.3360,  0.1867, -2.3957],
        [ 3.5087, -2.3970, -3.5488],
        [ 4.4981, -2.3366, -3.8062],
        [-4.9746, -0.4672,  4.9777],
        [-3.4076,  4.4987, -2.3804]], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [22]:
output.argmax(dim=1)

tensor([1, 0, 1, 0, 0, 0, 2, 1], device='cuda:0')

In [23]:
batch_length.shape

torch.Size([8])

In [24]:
label

tensor([1, 0, 1, 0, 0, 0, 2, 1], device='cuda:0')

In [25]:
loss_function(output,label)

tensor(0.0787, device='cuda:0', grad_fn=<NllLossBackward>)

In [26]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [tweet.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor, torch.tensor(1, dtype = torch.int64).unsqueeze(0))
    _, idx = torch.max(m(output[0].reshape(1,3)), dim=1)
#     max_preds = preds[0].argmax(dim = 1)
#     print(max_preds)
    return idx

In [27]:
class_label.vocab.stoi

defaultdict(None, {'1': 0, '0': 1, '2': 2})

In [49]:
pred_class = predict_class(model, sen)
itol = {'0': 'Hate Speech', '1': 'Offensive', '2': 'Neither'}
print(f'Predicted class is: {pred_class} = {itol[class_label.vocab.itos[pred_class]]}')

Predicted class is: tensor([1], device='cuda:0') = Hate Speech


In [51]:
from tqdm import tqdm
total_acc=0.0
m = nn.LogSoftmax(dim = 1)
for i, batch in enumerate(tqdm(test_iterator)):
    (feature, batch_length), label = batch.text, batch.c

    optimizer.zero_grad()
    output = model(feature, batch_length).squeeze()
    acc=categorical_accuracy(output,label)
    total_acc+=acc.item() 
print(f"accuracy = {total_acc/len(test_iterator)}")

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 339.20it/s]

accuracy = 0.7489451479308212


In [ ]:
h, o, n = 0,0,0
for example in train_data:
    if(int(example.c)==0):
        h += 1
    elif(int(example.c)==1):
        o += 1
    elif(int(example.c)==2):
        n += 1

In [ ]:
h,o,n

In [ ]:
h, o, n

In [ ]:
print(class_label.vocab.stoi)

In [42]:
sentence = test_data[100].text

In [43]:
sen = ""
for word in sentence:
    sen = sen +" " + word

In [44]:
sen

' "@rishaawilliams: @treshon_scrooge @_nevertrustt ol bitch ass" http://t.co/amcoiny0wg'

In [46]:
sentence

['"@rishaawilliams:',
 '@treshon_scrooge',
 '@_nevertrustt',
 'ol',
 'bitch',
 'ass"',
 'http://t.co/amcoiny0wg']

In [48]:
test_data[100].c

'1'